#Solution 2

In [1]:
!apt update && apt install cuda-11-8

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,755 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Hit:13 https://ppa.lau

In [2]:
!git clone https://github.com/dialogue-evaluation/RuNNE.git

Cloning into 'RuNNE'...
remote: Enumerating objects: 1620, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1620 (delta 17), reused 15 (delta 15), pack-reused 1598
Receiving objects: 100% (1620/1620), 2.59 MiB | 16.58 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [3]:
!git clone https://github.com/bond005/runne_contrastive_ner.git

Cloning into 'runne_contrastive_ner'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 81 (delta 26), reused 69 (delta 17), pack-reused 0
Receiving objects: 100% (81/81), 1.21 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [4]:
!cd runne_contrastive_ner && python -m pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.9 MB/s eta 0:00:00


In [5]:
!gdown 18v_rPQO0fPi76xmHrVr7KxZZh52Gx7aN

Downloading...
From: https://drive.google.com/uc?id=18v_rPQO0fPi76xmHrVr7KxZZh52Gx7aN
To: /content/data.zip
100% 963k/963k [00:00<00:00, 135MB/s]


In [6]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/dev.jsonl          
  inflating: data/ners.txt           
  inflating: data/test.jsonl         
  inflating: data/train.jsonl        


In [7]:
!python runne_contrastive_ner/split_data.py data/train.jsonl data/train.jsonl data/val.jsonl

There are 29 named entity classes:
  PERSON            5119
  PROFESSION        5030
  ORGANIZATION      4086
  EVENT             3335
  DATE              2689
  COUNTRY           2510
  CITY              1261
  NUMBER            1107
  AGE                657
  ORDINAL            614
  NATIONALITY        437
  FACILITY           424
  STATE_OR_PROVINCE  412
  AWARD              403
  LAW                401
  LOCATION           314
  IDEOLOGY           273
  WORK_OF_ART        270
  PRODUCT            245
  CRIME              221
  DISEASE            220
  TIME               182
  MONEY              179
  DISTRICT           103
  PENALTY             92
  RELIGION            89
  PERCENT             68
  LANGUAGE            54
  FAMILY              24

For training:
  PERSON            4239
  PROFESSION        4244
  ORGANIZATION      3366
  EVENT             2749
  DATE              2274
  COUNTRY           2089
  CITY              1048
  NUMBER             931
  AGE                551


In [8]:
#To remove the following error:
#ValueError: The entity (WORK_OF_ART, 144, 166) is wrong! It is located in more than a single sentence. More probably sentence is "Калуга, дом на углу Достоевского и Московской
#10 февраля 2019 года, в Калуге представили проект Art in Art."

#Because the name of the art is "Art in Art. Reflection"
#There is a dot in the name which is considered as the end of the sentence

#So we fix by replacing the dot



with open("data/train.jsonl", 'r', encoding='utf-8') as file:
    content = file.read()

modified_content = content.replace("Art in Art. Reflection", "Art in Art: Reflection")

with open("data/train.jsonl", 'w', encoding='utf-8') as file:
    file.write(modified_content)

In [29]:
!python runne_contrastive_ner/prepare_trainset.py \
    data/train.jsonl \
    data/train_siamese_dprubert_128.pkl \
    data/ners.txt \
    siamese \
    128 \
    DeepPavlov/rubert-base-cased \
    10000

Number of texts is 441.
  4% 16/441 [00:00<00:05, 76.88it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 0, 18), ('PROFESSION', 20, 45), ('PROFESSION', 28, 45), ('CITY', 46, 56), ('ORGANIZATION', 46, 77)] can have errors. The entity ('ORGANIZATION', 46, 77) is inexactly found in the text "Екатерина Хотинова, старший научный сотрудник сочинского Художественного музея.". 77 != 78
  warnings.warn(warn_msg)
 11% 47/441 [00:00<00:05, 67.95it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 0, 5), ('ORGANIZATION', 24, 43), ('COUNTRY', 39, 43), ('ORGANIZATION', 46, 72), ('COUNTRY', 68, 71), ('ORGANIZATION', 74, 77)] can have errors. The entity ('COUNTRY', 68, 71) is inexactly found in the text "Кейта выдвинула партия "Объединение за Мали" (Rassemblement pour le Mali, RPM), руководителем которой он является.". 71 != 72
  warnings.warn(warn_msg)
 18% 79/441 [00:01<00:04, 7

In [30]:
!python runne_contrastive_ner/prepare_trainset.py \
    data/val.jsonl \
    data/val_siamese_dprubert_128.pkl \
    data/ners.txt \
    siamese \
    128 \
    DeepPavlov/rubert-base-cased \
    1000

Number of texts is 78.
 45% 35/78 [00:01<00:01, 39.14it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 45, 60), ('IDEOLOGY', 64, 80), ('COUNTRY', 74, 81), ('IDEOLOGY', 84, 97)] can have errors. The entity ('IDEOLOGY', 64, 80) is inexactly found in the text "В своём послании он подверг критике политику Дональда Трампа за поддержку Израиля и консерваторов за их «недостаточно агрессивную» политику.". 80 != 81
  warnings.warn(warn_msg)
100% 78/78 [00:01<00:00, 41.62it/s]
Number of sentences is 1179.
Iter 1: there are 5 very long texts!
Number of sentences after cutting is 1184.
  0% 0/1000 [00:00<?, ?it/s]/content/runne_contrastive_ner/trainset_building/trainset_building.py:266: UserWarning: The pair FAMILY-FAMILY is not found.
  warnings.warn(warn_msg)
100% 1000/1000 [00:00<00:00, 133080.69it/s]
All 1000 samples are built.


In [11]:
!mkdir model

In [12]:
#2024-04-28 11:10:53.255856: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
#Warning: Could not find Tensor-RT
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 44.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninst

In [13]:
!pip install tensorflow-addons

In [36]:
!python runne_contrastive_ner/train.py \
    data/train_siamese_dprubert_128.pkl \
    data/val_siamese_dprubert_128.pkl \
    model/runne_siamese_rubert_deeppavlov \
    siamese \
    16 \
    DeepPavlov/rubert-base-cased \
    from-pytorch

2024-04-28 18:23:01.742132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 18:23:02.696594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorf

In [15]:
!python runne_contrastive_ner/prepare_trainset.py \
    data/train.jsonl \
    data/train_ner_dprubert_128.pkl \
    data/ners.txt \
    ner \
    128 \
    DeepPavlov/rubert-base-cased

Number of texts is 441.
  4% 16/441 [00:00<00:09, 43.01it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 0, 18), ('PROFESSION', 20, 45), ('PROFESSION', 28, 45), ('CITY', 46, 56), ('ORGANIZATION', 46, 77)] can have errors. The entity ('ORGANIZATION', 46, 77) is inexactly found in the text "Екатерина Хотинова, старший научный сотрудник сочинского Художественного музея.". 77 != 78
  warnings.warn(warn_msg)
 11% 48/441 [00:01<00:12, 31.54it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 0, 5), ('ORGANIZATION', 24, 43), ('COUNTRY', 39, 43), ('ORGANIZATION', 46, 72), ('COUNTRY', 68, 71), ('ORGANIZATION', 74, 77)] can have errors. The entity ('COUNTRY', 68, 71) is inexactly found in the text "Кейта выдвинула партия "Объединение за Мали" (Rassemblement pour le Mali, RPM), руководителем которой он является.". 71 != 72
  warnings.warn(warn_msg)
 19% 82/441 [00:02<00:10, 3

In [16]:
!python runne_contrastive_ner/prepare_trainset.py \
    data/val.jsonl \
    data/val_ner_dprubert_128.pkl \
    data/ners.txt \
    ner \
    128 \
    DeepPavlov/rubert-base-cased

Number of texts is 78.
 47% 37/78 [00:00<00:00, 60.73it/s]/content/runne_contrastive_ner/data_processing/tokenization.py:406: UserWarning: The annotation [('PERSON', 45, 60), ('IDEOLOGY', 64, 80), ('COUNTRY', 74, 81), ('IDEOLOGY', 84, 97)] can have errors. The entity ('IDEOLOGY', 64, 80) is inexactly found in the text "В своём послании он подверг критике политику Дональда Трампа за поддержку Израиля и консерваторов за их «недостаточно агрессивную» политику.". 80 != 81
  warnings.warn(warn_msg)
100% 78/78 [00:01<00:00, 60.72it/s]
Number of sentences is 1179.
Iter 1: there are 5 very long texts!
Number of sentences after cutting is 1184.

X.shape = (1184, 128)
y[0].shape = (1184, 128, 5)
y[1].shape = (1184, 128, 5)
y[2].shape = (1184, 128, 5)
y[3].shape = (1184, 128, 5)
y[4].shape = (1184, 128, 5)
y[5].shape = (1184, 128, 5)
y[6].shape = (1184, 128, 5)
y[7].shape = (1184, 128, 5)
y[8].shape = (1184, 128, 5)
y[9].shape = (1184, 128, 5)
y[10].shape = (1184, 128, 5)
y[11].shape = (1184, 128

In [ ]:
#This commands arguments are wrong in the Github

In [17]:
!python runne_contrastive_ner/train.py \
    data/train_ner_dprubert_128.pkl \
    data/val_ner_dprubert_128.pkl \
    model/runne_ner \
    ner \
    16 \
    model/runne_siamese_rubert_deeppavlov \
    from-tensorflow \
    data/ners.txt

2024-04-28 16:55:55.101140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 16:55:56.531078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorf

In [18]:
#Fixing typo

with open("data/test.jsonl", 'r', encoding='utf-8') as file:
    content = file.read()

modified_content = content.replace("senences", "sentences")

with open("data/test.jsonl", 'w', encoding='utf-8') as file:
    file.write(modified_content)

In [19]:
!mkdir submission

In [ ]:
!pip install transformers==4.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0


In [ ]:
!pip install tokenizers==0.11.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.0
    Uninstalling tokenizers-0.12.0:
      Successfully uninstalled tokenizers-0.12.0


In [28]:
!python runne_contrastive_ner/recognize.py \
    data/test.jsonl \
    model/runne_ner \
    submission/test.jsonl

2024-04-28 17:30:02.595026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 17:30:03.525518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorf

In [ ]:
!nvidia-smi

Sun Apr 28 12:58:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
'''
!sudo apt --purge remove "cublas*" "cuda*"
!sudo apt --purge remove "nvidia*"
!rm -rf /usr/local/cuda*
!sudo apt-get autoremove && sudo apt-get autoclean

!sudo apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.7.0/local_installers/cuda-repo-ubuntu1804-11-7-local_11.7.0-515.43.04-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-7-local_11.7.0-515.43.04-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu1804-11-7-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda-11-7
'''